In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
import os

#imports from mmy photometry code
from ptsrc_photometry_photutils import run_dao_star_finder,calculate_bkgrnd,run_aperture_photometry
from uvis_contam_tools import make_PAMcorr_image


In [3]:
#F218W files, LAcosmic ran on flts to remove rays

f218w_files = glob.glob('/grp/hst/wfc3t/cshanahan/UVIS_contam_monitor/data/data/GRW70/F218W/flt_cleans/*.fits')

#seperate files by detector quadrant 
amp_sorted_files=([],[],[],[]) #A, B, C, D

sub_amp_dict={'UVIS1-C512A-SUB':(1,'A'),'UVIS1-C512B-SUB':(1,'B'),\
           'UVIS2-C512C-SUB':(2,'C'), 'UVIS2-C512D-SUB':(2,'D'),\
           'UVIS2-M512C-SUB':(2,'C')}

for f in f218w_files:
    subarray = fits.open(f)[0].header['aperture']
    amp = sub_amp_dict[subarray][1]
    
    for i, val in enumerate(['A','B','C','D']):
        if amp == val:
            amp_sorted_files[i].append(f)

In [4]:
aperture_size = 10.

In [5]:
#process amp A data...first 10
ampA = amp_sorted_files[0][0:10]


for filee in ampA:
    hdu_list = fits.open(filee)
    filename = os.path.basename(filee)
    
    #find source in image
    coo_tab = run_dao_star_finder(filee,filename,hdu_list)
    
    #sky subtract
    back,backrms = calculate_bkgrnd(hdu_list, filename, coo_tab)
    #PAM
    make_PAMcorr_image(filee,outfile=filee+'.pamcorr', extension=0)
    pam_filee = filee+'.pamcorr'
    pam_hdu_list = fits.open(pam_filee)
    pam_header = pam_hdu_list[0].header
    pam_sci = pam_hdu_list[0].data
    
    
    
   
